# Analyse des Datasets

In [22]:
import pandas as pd

# Charger les datasets
matches = pd.read_csv("datasets/matches.csv")
joueurs = pd.read_csv("datasets/joueurs_grands_championnats.csv")

# Afficher les premières lignes
print("Aperçu de matches.csv :")
print(matches.head())

print("\nAperçu de joueurs_grands_championnats.csv :")
print(joueurs.head())

# Vérifier les infos générales
print("\nInfos matches.csv :")
print(matches.info())

print("\nInfos joueurs_grands_championnats.csv :")
print(joueurs.info())
print(joueurs['Date de Naissance'].head())

mois_fr_en = {
    "jan.": "jan", "févr.": "feb", "mars": "mar", "avr.": "apr", "mai": "may", "juin": "jun",
    "juil.": "jul", "août": "aug", "sept.": "sep", "oct.": "oct", "nov.": "nov", "déc.": "dec"
}

# Fonction pour remplacer les mois en français
def convertir_mois(date_str):
    for fr, en in mois_fr_en.items():
        date_str = date_str.replace(fr, en)
    return date_str


Aperçu de matches.csv :
         Date       Équipe 1  Score 1  Score 2        Équipe 2 Championnat  \
0  2023-08-11        Almeria        0        2  Rayo Vallecano     la-liga   
1  2023-08-11        Sevilla        1        2        Valencia     la-liga   
2  2023-08-12  Real Sociedad        1        1          Girona     la-liga   
3  2023-08-12     Las Palmas        1        1        Mallorca     la-liga   
4  2023-08-12  Athletic Club        0        2     Real Madrid     la-liga   

   Saison            Ville  Temp Max  Temp Min  Précipitations  
0    2023          Almería      35.9      26.5             0.0  
1    2023          Séville      44.6      24.9             0.0  
2    2023  Saint-Sébastien      24.4      17.3             0.5  
3    2023       Las Palmas      41.7      26.4             0.0  
4    2023           Bilbao      24.5      18.0             2.2  

Aperçu de joueurs_grands_championnats.csv :
  Championnat  Saison                 Club                Joueur  \
0   

In [20]:
col_club = "Club"
col_saison = "Saison"
col_age = "Age"
col_valeur = "Valeur Marchande"


from dateutil import parser

joueurs['Date de Naissance'] = joueurs['Date de Naissance'].apply(lambda x: parser.parse(x, dayfirst=True) if pd.notnull(x) else pd.NaT)

print(joueurs['Date de Naissance'].head())
joueurs['Date de Naissance'] = pd.to_datetime(joueurs['Date de Naissance'], format='%d %b %Y', errors='coerce')

print(joueurs['Date de Naissance'].head())

joueurs['annee_naissance'] = joueurs['Date de Naissance'].dt.year


# Calcul de l'âge en fonction de l'année actuelle
joueurs['Age'] = col_saison - joueurs['Date de Naissance'].dt.year
joueurs['Saison'] = joueurs[col_saison].astype(int)

# Maintenant, tu peux regrouper et calculer les moyennes comme tu le souhaites
features_clubs = joueurs.groupby([col_club, col_saison]).agg(
    Age_Moyen=('Age', 'mean'),
    Valeur_Marchande_Moyenne=(col_valeur, 'mean')
).reset_index()

print(features_clubs.head())



ParserError: Unknown string format: 25 févr. 1999 (23)

In [ ]:
import pandas as pd
from dateutil import parser
import re
import numpy as np

col_club = "Club"
col_saison = "Saison"
col_age = "Age"
col_valeur = "Valeur Marchande"
col_date_naissance = "Date de Naissance"

# Mapping des mois français vers anglais
mois_fr_en = {
    "janv.": "Jan", "févr.": "Feb", "mars": "Mar", "avr.": "Apr", "mai": "May", "juin": "Jun",
    "juil.": "Jul", "août": "Aug", "sept.": "Sep", "oct.": "Oct", "nov.": "Nov", "déc.": "Dec"
}

# Fonction pour nettoyer la date (supprimer l'âge entre parenthèses et traduire les mois)
def nettoyer_date(date_str):
    if pd.isna(date_str) or date_str in ["N/A", "NaN", "None"]:  # Gérer les valeurs non valides
        return np.nan
    date_str = re.sub(r"\s*\(.*\)", "", date_str)  # Supprimer l'âge entre parenthèses
    for fr, en in mois_fr_en.items():  # Remplacer les mois français par anglais
        date_str = date_str.replace(fr, en)
    return date_str

# Nettoyer et traduire les dates
joueurs[col_date_naissance] = joueurs[col_date_naissance].astype(str).apply(nettoyer_date)

# Appliquer la conversion en datetime avec gestion des erreurs
joueurs[col_date_naissance] = pd.to_datetime(joueurs[col_date_naissance], format='%d %b %Y', errors='coerce')

# Extraire l'année de naissance
joueurs['annee_naissance'] = joueurs[col_date_naissance].dt.year

# Vérification du type de `col_saison`
joueurs[col_saison] = pd.to_numeric(joueurs[col_saison], errors='coerce')  # Convertir en numérique

# Calcul de l'âge
joueurs[col_age] = joueurs[col_saison] - joueurs['annee_naissance']



# Convertir en float pour éviter les erreurs dans `.agg()`
joueurs[col_age] = pd.to_numeric(joueurs[col_age], errors='coerce')
def nettoyer_valeur(val):
    if pd.isna(val) or val in ["N/A", "NaN", "None"]:  # Gérer les valeurs manquantes
        return np.nan
    val = str(val).lower().replace(" ", "").replace(",", ".")  # Supprimer espaces et remplacer , par .
    val = re.sub(r"[€mio.]", "", val)  # Supprimer les symboles €
    if "m" in val:
        return float(val.replace("m", "")) * 1e6
    elif "k" in val:
        return float(val.replace("k", "")) * 1e3
    try:
        return float(val)  # Conversion classique
    except ValueError:
        return np.nan  # Si impossible à convertir

# Appliquer la fonction de nettoyage
joueurs[col_valeur] = joueurs[col_valeur].apply(nettoyer_valeur)

# Vérifier que tout est bien numérique
print(joueurs[col_valeur].unique())  # Vérifie les valeurs uniques après nettoyage

joueurs[col_valeur] = pd.to_numeric(joueurs[col_valeur], errors='coerce')

# Regroupement et calcul des moyennes
features_clubs = joueurs.groupby([col_club, col_saison]).agg(
    Age_Moyen=(col_age, 'mean'),
    Valeur_Marchande_Moyenne=(col_valeur, 'mean')
).reset_index()

print(features_clubs.head())


SyntaxError: invalid syntax (<unknown>, line 1)